In [1]:
"""
Imports Here
"""
from dataset import trainloader
from dataset import testloader
from dataset import trainaccloader
from dataset import trainshape
from dataset import testshape

/Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 6): Library not loaded: @rpath/libpng16.16.dylib
  Referenced from: /Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Reason: Incompatible library version: image.so requires version 56.0.0 or later, but libpng16.16.dylib provides version 54.0.0'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
from model import VisionTransformer
from model import Attention
from model import TransBlock

In [30]:
TRAIN_LOSS = []
TRAIN_ACC = []

def train(model, dataloader, accloader, lossfunc, optimiser, lr=0.1, momentum=0.9, batchsize=16, nepochs=10):
    device = next(model.parameters()).device # check what device the net parameters are on
    
    """training"""
    for i in range(nepochs): # for each epoch
        epoch_loss = 0
        model.train()
        n_batches = 0
        time1 = time.time()
        for (x, y) in dataloader: # for each mini-batch
            optimiser.zero_grad()
            loss = lossfunc(model.forward(x), y)
            loss.backward()
            optimiser.step()
            epoch_loss += loss
            n_batches += 1
        time2 = time.time()
        print("Done an epoch", time2-time1)
        epoch_loss /= n_batches
    
        """evaluating"""
        model.eval()
        accuracy = test(model, accloader)

        """get performance"""
        TRAIN_LOSS.append(epoch_loss.item())
        TRAIN_ACC.append(accuracy)

def test(model, dataloader):
    with torch.no_grad(): # disable automatic gradient computation for efficiency
        device = next(model.parameters()).device
        
        """make predictions"""
        pcls = []
        items = 0
        time1=time.time()
        for x, y in dataloader:
            x = x.to(device)
            pcls.append(abs(y.cpu()-torch.max(model(x), 1)[1].cpu()))
            items += 1
        time2 = time.time()
        print("found accuracy in:", time2-time1)

        """get accuracy"""
        pcls = torch.cat(pcls) # concat predictions on the mini-batches
        accuracy = 1 - (pcls.sum().float() / items)
        print("accuracy:", accuracy)
        return accuracy

In [31]:
batchsize=16
N, Np, P = trainshape()
model = VisionTransformer(inputsize=(batchsize, Np, P), embed=128, fflscale=2, nblocks=4)

In [32]:
import time
import torch
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimiser = optim.AdamW(model.parameters(), lr=1e-4)

In [33]:
start = time.time()
train(model, trainloader(batchsize=batchsize), trainaccloader(), criterion, optimiser, nepochs=10)
end = time.time()
print("training time: " end-start)
test(model, testloader())

loop time:  3.66955304145813
loop time:  4.265578031539917
loop time:  5.169572830200195
loop time:  4.732528924942017


KeyboardInterrupt: 

In [35]:
print(TRAIN_LOSS)
print(TRAIN_ACC)

[]
[]
